In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

import matplotlib.pyplot as plt, matplotlib
import numpy as np, healpy as hp, h5py
from scipy import stats, special, linalg, optimize
import scipy
import tqdm

from pylab import cm

# Plotting modules
import matplotlib
from pylab import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', labelsize=16)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('legend',fontsize=16)
plt.rc('font',size=16)

In [3]:
import warnings
warnings.filterwarnings("ignore", message="You are modifying the state of a globally registered colormap.")
warnings.filterwarnings("ignore", message="Passing parameters norm and vmin/vmax simultaneously is deprecated")
warnings.filterwarnings("ignore", message="divide by zero encountered")
warnings.filterwarnings("ignore", message="overflow encountered in exp")

In [4]:
from numba import njit

## Test sample

In [5]:
import sys, h5py, numpy as np, scipy.stats, healpy as hp, tqdm
from SelectionFunctionChisel import Chisel

In [6]:
np.arange(7,22.1,1)

array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.,
       20., 21., 22.])

In [137]:
eps=1e-10

lengthscale = 0.3
j=[-1,0,1,2,3,4]
nside=8

M_bins = np.arange(5,22.1,1)
M = M_bins.shape[0]-1
C = 1

lengthscale = 0.3
lengthscale_m = lengthscale/(M_bins[1]-M_bins[0])
lengthscale_c = 1.

file_root = f"chisquare_j{str(j).replace(' ','')}_nside{nside}_M{M}_C{C}_l{lengthscale}"
basis_options = {'needlet':'chisquare', 'j':j, 'B':2.0, 'p':1.0, 'wavelet_tol':1e-2}

nside_original = 128
resize = int((nside_original/nside)**2+0.1)

In [138]:
box={};
with h5py.File('/data/asfe2/Projects/astrometry/gaia3_astcounts_arr_hpx128.h', 'r') as hf:
    box['n'] = hf['n'][...]
    box['k'] = hf['k'][...]
    M_bins = hf['magbins'][...]
print("Mag bins:", np.linspace(M_bins[0], M_bins[-1], M+1))

Mag bins: [ 5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22.]


In [139]:
# Import chisel
chisel = Chisel(box['k'], box['n'],
                basis_options,file_root,
                axes = ['magnitude','colour','position'],
                nest = True,
                lengthscale_m = lengthscale_m,
                lengthscale_c = lengthscale_c,
                M = M,
                C = C,
                nside = nside,
                sparse = True,
                pivot = True,
                mu = 0.0,
                sigma = [-0.81489922, -2.55429039],
                spherical_basis_directory='/data/asfe2/Projects/astrometry/SphericalWavelets/',
                stan_output_directory='/data/asfe2/Projects/astrometry/StanOutput/'
                )

Spherical basis file does not exist, generating... (this may take some time!)
Working on order -1.
Working on order 0.
100%|██████████| 12/12 [00:00<00:00, 17.62it/s]
Working on order 1.
100%|██████████| 48/48 [00:00<00:00, 6364.85it/s]
Working on order 2.
100%|██████████| 192/192 [00:00<00:00, 1725.82it/s]
Working on order 3.
100%|██████████| 768/768 [00:00<00:00, 1775.08it/s]
Working on order 4.
100%|██████████| 3072/3072 [00:02<00:00, 1316.50it/s]


INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/andy/Documents/Research/Projects/EDR3_selection/astrometry/SelectionFunctionToolkit/StanModels/wavelet_magnitude_colour_position_sparse


Spherical basis file loaded
17 17
1 1
88.58% sparsity
0.00% sparsity
Incrementing wavelet_u
Incrementing wavelet_v
Incrementing cholesky_v_m
Incrementing cholesky_u_m
Incrementing cholesky_v_c
Incrementing cholesky_u_c


In [304]:
#@njit
def matrix_multiply(k, n, P, x, b, wavelet_w, wavelet_v, wavelet_u):
    
    lnL = 0.
    lnL_grad = np.zeros(x.shape)
    
    # Iterate over pixels
    for ipix in range(P):
#         # Iterate over modes which are not sparsified in Y
# #         imin = wavelet_u[ipix]
# #         imax = wavelet_u[ipix+1]
# #         for iY, iS in enumerate(wavelet_v[imin:imax]):
# #             x[ipix] += b[int(iS-1+eps)] * wavelet_w[int(wavelet_u[ipix]+iY-1+eps)]

            
        lnL += np.sum( -k[ipix]*np.log(1+np.exp(-x[ipix])) - (n[ipix]-k[ipix])*np.log(1+np.exp(x[ipix])) )
        
        # Gradient wrt p --> dlnL/dp
        lnL_grad[ipix]  = (k[ipix] - n[ipix]/(1+np.exp(-x[ipix])))*(1+np.exp(x[ipix]))*(1+np.exp(-x[ipix]))
        # Jacobian p-x   --> dp/dx
        lnL_grad[ipix] *= np.exp(-x[ipix])/(1+np.exp(-x[ipix]))**2
        
    return lnL, lnL_grad

In [305]:
P = chisel.nside_to_npix(chisel.nside)
x = np.zeros((P, M, C))+0.5
b = np.zeros((chisel.S, M, C))

In [282]:
P = 10
k_test = np.zeros(P, dtype=int) + 50
n_test = np.zeros(P, dtype=int) + 100
x_test = np.zeros(P, dtype=float)#-0.01

foo = lambda x: matrix_multiply(k_test, n_test, \
                P, x, b, \
                chisel.stan_input['wavelet_w'], chisel.stan_input['wavelet_v'], chisel.stan_input['wavelet_u'])

In [283]:
grad = lambda x: foo(x)[1]
model = lambda x: foo(x)[0]

scipy.optimize.approx_fprime(x_test, model, 1e-8), grad(x_test)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [306]:
P = 10
x_test = np.zeros((P, M, C))+0.5

k_test = np.moveaxis(chisel.stan_input['k'][:,:,:P], -1,0).astype(np.int64)
n_test = np.moveaxis(chisel.stan_input['n'][:,:,:P], -1,0).astype(np.int64)

foo = lambda x: matrix_multiply(k_test, n_test, \
                P, x, b, \
                chisel.stan_input['wavelet_w'], chisel.stan_input['wavelet_v'], chisel.stan_input['wavelet_u'])

In [307]:
grad = lambda x: foo(x.reshape((P,M,C)))[1].flatten()
model = lambda x: foo(x.reshape((P,M,C)))[0]

scipy.optimize.approx_fprime(x_test.flatten(), model, 1e-8), grad(x_test.flatten())

(array([ 4.17232511e+01,  1.47521495e+02,  4.02331350e+02,  1.11460685e+03,
         2.92062758e+03,  7.33286139e+03,  1.69217586e+04,  3.84911893e+04,
         8.15317031e+04,  1.64858996e+05,  3.24936209e+05,  6.14975390e+05,
         1.09386444e+06,  1.78058147e+06,  2.61979847e+06,  2.16462462e+06,
        -9.95807345e+05,  2.53319739e+01,  8.79168506e+01,  2.45869158e+02,
         6.39259812e+02,  1.48117542e+03,  3.39895485e+03,  7.04973933e+03,
         1.41009688e+04,  2.75865196e+04,  5.03644345e+04,  8.16851850e+04,
         1.20827555e+05,  1.68183445e+05,  2.33279167e+05,  3.30106912e+05,
         2.80322133e+05, -2.62530147e+05,  2.98023222e+01,  8.34465023e+01,
         2.23517417e+02,  5.66244123e+02,  1.33514404e+03,  2.95042990e+03,
         6.03795049e+03,  1.20013952e+04,  2.34514474e+04,  4.12195919e+04,
         6.69360157e+04,  9.98854632e+04,  1.41613185e+05,  2.00162827e+05,
         2.87516414e+05,  2.10653244e+05, -1.83168053e+05,  4.61935995e+01,
         1.3

In [94]:
@njit
def matrix_multiply(k, n, P, b, wavelet_w, wavelet_v, wavelet_u, lnL_grad, x):
    
    lnL = 0.
    lnL_grad_local = np.zeros(lnL_grad.shape)
    lnL_grad *= 0.
    #x = np.zeros(b.shape[1:])
    
    # Iterate over pixels
    for ipix in range(P):
        x *= 0.
        lnL_grad_local *= 0.
        # Iterate over modes which are not sparsified in Y
        imin = wavelet_u[ipix]
        imax = wavelet_u[ipix+1]
        iY = 0
        for iS in wavelet_v[imin:imax]:
            
            x += b[iS-1] * wavelet_w[wavelet_u[ipix]-1+iY]
            
            lnL_grad_local[iS-1] += wavelet_w[int(wavelet_u[ipix]+iY-1+eps)]
            
            iY += 1
            
        exp_x = np.exp(x)
        
        lnL += np.sum( -k[ipix]*np.log(1+1/exp_x) - (n[ipix]-k[ipix])*np.log(1+exp_x) )
        lnL_grad += lnL_grad_local * (k[ipix] - n[ipix]/(1+1/exp_x))
        
    return lnL, lnL_grad

In [95]:
P = 10
b_test = np.zeros((chisel.S, M, C))+0.01

x_test = np.zeros((P, M, C))#+0.5
k_test = np.moveaxis(chisel.stan_input['k'][:,:,:P], -1,0).astype(np.int64)
n_test = np.moveaxis(chisel.stan_input['n'][:,:,:P], -1,0).astype(np.int64)
lnL_grad = np.zeros(b_test.shape)
lnL_grad_local = np.zeros(b_test.shape)
x = np.zeros(b_test.shape[1:])

foo = lambda b: matrix_multiply(k_test, n_test, \
                P, b, \
                chisel.stan_input['wavelet_w'], chisel.stan_input['wavelet_v'], chisel.stan_input['wavelet_u'],
                               lnL_grad, x)

In [96]:
grad = lambda b: foo(b.reshape((chisel.S,M,C)))[1].flatten()
model = lambda b: foo(b.reshape((chisel.S,M,C)))[0]

scipy.optimize.approx_fprime(b_test.flatten(), model, 1e-5), grad(b_test.flatten())

(array([-2.49445438e+00, -7.91996717e+00, -2.31742859e+01, ...,
        -6.16494849e+04, -5.60341284e+04,  1.82281211e+04]),
 array([-2.49339577e+00, -7.92016667e+00, -2.31756236e+01, ...,
        -6.16494844e+04, -5.60341272e+04,  1.82281198e+04]))

In [380]:
grad = lambda b: foo(b.reshape((chisel.S,M,C)))[1].flatten()
model = lambda b: foo(b.reshape((chisel.S,M,C)))[0]

scipy.optimize.approx_fprime(b_test.flatten(), model, 1e-5), grad(b_test.flatten())

(array([-2.49445438e+00, -7.91996717e+00, -2.31742859e+01, ...,
        -6.16494849e+04, -5.60341284e+04,  1.82281211e+04]),
 array([-2.49339577e+00, -7.92016667e+00, -2.31756236e+01, ...,
        -6.16474130e+04, -5.60311298e+04,  1.82287245e+04]))

In [97]:
%timeit model(b_test.flatten())

618 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [68]:
%timeit model(b_test.flatten())

915 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%timeit model(b_test.flatten())

1.14 ms ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit model(b_test.flatten())

1.1 ms ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [375]:
%timeit model(b_test.flatten())

1.11 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [368]:
%timeit model(b_test.flatten())

1.74 ms ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Run full test

In [140]:
@njit
def matrix_multiply(k, n, P, b, wavelet_w, wavelet_v, wavelet_u, lnL_grad, x):
    
    lnL = 0.
    lnL_grad *= 0.
    lnL_grad_local = np.zeros(lnL_grad.shape)
    
    # Iterate over pixels
    for ipix in range(P):
        x *= 0.
        lnL_grad_local *= 0.
        # Iterate over modes which are not sparsified in Y
        imin = wavelet_u[ipix]
        imax = wavelet_u[ipix+1]
        iY = 0
        for iS in wavelet_v[imin:imax]:
            
            x += b[iS-1] * wavelet_w[wavelet_u[ipix]-1+iY]
            
            lnL_grad_local[iS-1] += wavelet_w[int(wavelet_u[ipix]+iY-1+eps)]
            
            iY += 1
            
        exp_x = np.exp(x)
            
        lnL_grad += lnL_grad_local * (k[ipix] - n[ipix]/(1+1/exp_x))
        lnL += np.sum( -k[ipix]*np.log(1+1/exp_x) - (n[ipix]-k[ipix])*np.log(1+exp_x) )
        
    return lnL, lnL_grad

In [141]:
P = 12*nside**2
b = np.zeros((chisel.S, M, C))

k = np.moveaxis(chisel.stan_input['k'], -1,0).astype(np.int64)
n = np.moveaxis(chisel.stan_input['n'], -1,0).astype(np.int64)
lnL_grad = np.zeros(b.shape)
x = np.zeros(b.shape[1:])

In [142]:
def function(b):

    lnL, grad = matrix_multiply(k, n, P, b.reshape((chisel.S,M,C)), \
                                    chisel.stan_input['wavelet_w'], 
                                    chisel.stan_input['wavelet_v'], 
                                    chisel.stan_input['wavelet_u'],
                                       lnL_grad, x)
    
    return -lnL, -grad.flatten()

In [143]:
grad = lambda b: function(b)[1]
model = lambda b: function(b)[0]

scipy.optimize.approx_fprime(b.flatten(), model, 1e-5), grad(b.flatten())

MemoryError: Unable to allocate 36.1 GiB for an array with shape (69581, 69581) and data type float64

In [144]:
%timeit model(b.flatten())

1.28 s ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
%timeit model(b.flatten())

132 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


46.7 ms ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
bounds = np.zeros((np.prod(b.shape), 2))
bounds[:,0] = -5
bounds[:,1] = 5

In [103]:
function(b.flatten())

(1251957722.7402508,
 array([ 1.02838376e+01,  3.59199228e+01,  9.65588766e+01, ...,
        -6.36946438e+06, -4.47684562e+06,  1.21183053e+06]))

In [104]:
res = scipy.optimize.minimize(function, b.flatten(), method='L-BFGS-B', jac=True, bounds=bounds)
res

In [105]:
res

      fun: 535348913.7851486
 hess_inv: <1037x1037 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.17764423e-01, -1.06123634e+00, -6.94044130e-01, ...,
       -9.91846948e+02,  5.00200710e+03, -1.79412172e+02])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 653
      nit: 605
     njev: 653
   status: 0
  success: True
        x: array([-0.65387882, -0.65884027, -0.66208827, ...,  5.        ,
        4.50639605, -0.1575282 ])

In [107]:
scipy.optimize.approx_fprime(res['x'], model, 1e-5), grad(res['x'])

(array([-4.11272049e-01, -1.07884407e+00, -7.03334808e-01, ...,
        -9.91624594e+02,  5.00510931e+03, -1.79392099e+02]),
 array([-4.17764423e-01, -1.06123634e+00, -6.94044130e-01, ...,
        -9.91846948e+02,  5.00200710e+03, -1.79412172e+02]))

In [93]:
scipy.optimize.approx_fprime(res['x'], model, 1e-2), grad(res['x'])

(array([ 4.78332043e+00,  1.77561283e+01,  5.17903566e+01, ...,
        -3.72028802e+05, -9.32256607e+05,  3.44056878e+03]),
 array([ 4.77816735e+00,  1.77405169e+01,  5.17466468e+01, ...,
        -3.72569976e+05, -9.35734427e+05,  3.43023125e+03]))

## Stan model

In [ ]:
_model_file = 'wavelet_magnitude_colour_position'
_model_file += '_sparse' if self.sparse else ''

from cmdstanpy import CmdStanModel
self.stan_model = CmdStanModel(stan_file = self.stan_model_directory+_model_file+'.stan')

In [ ]:
self.stan_model.optimize(data = self.stan_input, iter = number_of_iterations, output_dir = self.stan_output_directory)

In [ ]:
data {
    int<lower=0> P;                       // number of pixels
    int<lower=0> M;                       // number of bins in magnitude space
    int<lower=0> M_subspace;              // number of inducing points in magnitude space
    int<lower=0> C;                       // number of bins in colour space
    int<lower=0> C_subspace;              // number of inducing points in colour space
    int<lower=0> S;                       // number of wavelets
    int wavelet_n;                        // sparse wavelets - number of nonzero elements
    vector[wavelet_n] wavelet_w;          // sparse wavelets - nonzero elements
    int wavelet_v[wavelet_n];             // sparse wavelets - columns of nonzero elements
    int wavelet_u[P+1];                   // sparse wavelets - where in w each row starts
    vector[S] mu;                         // mean of each wavelet
    vector[S] sigma;                      // sigma of each wavelet
    int k[M,C,P];                         // number of heads
    int n[M,C,P];                         // number of flips
    row_vector[M_subspace] cholesky_m[M]; // Cholesky factor in magnitude space
    vector[C_subspace] cholesky_c[C];     // Cholesky factor in colour space
}
parameters {
    matrix[M_subspace,C_subspace] z[S];
}
transformed parameters {

    vector[P] x[M,C]; // Probability in logit-space
    
    // Loop over magnitude and colour
    for (m in 1:M){
        for (c in 1:C){
            
            // Local variable
            vector[S] b;
                
            // Compute b
            for (s in 1:S){
                b[s] = mu[s] + sigma[s] * cholesky_m[m] * z[s] * cholesky_c[c];
            }
                
            // Compute x
            x[m,c] = csr_matrix_times_vector(P, S, wavelet_w, wavelet_v, wavelet_u, b);

        }  
    }
    
}
model {

    // Prior
    for (s in 1:S){
        to_vector(z[s]) ~ std_normal();
    }
    
    // Likelihood
    for (m in 1:M){
        for (c in 1:C){
            k[m,c] ~ binomial_logit(n[m,c], x[m,c]);
        }
    }
    
}


In [ ]:
data {
    int<lower=0> P;                       // number of pixels
    int<lower=0> M;                       // number of bins in magnitude space
    int<lower=0> M_subspace;              // number of inducing points in magnitude space
    int<lower=0> C;                       // number of bins in colour space
    int<lower=0> C_subspace;              // number of inducing points in colour space
    int<lower=0> S;                       // number of harmonics
    int wavelet_n;                        // sparse wavelets - number of nonzero elements
    vector[wavelet_n] wavelet_w;          // sparse wavelets - nonzero elements
    int wavelet_v[wavelet_n];             // sparse wavelets - columns of nonzero elements
    int wavelet_u[P+1];                   // sparse wavelets - where in w each row starts
    vector[S] mu;                         // mean of each wavelet
    vector[S] sigma;                      // sigma of each wavelet
    int k[M,C,P];                         // number of heads
    int n[M,C,P];                         // number of flips
    int cholesky_n_m;                     // sparse cholesky in magnitude - number of nonzero elements
    row_vector[cholesky_n_m] cholesky_w_m;// sparse cholesky in magnitude - nonzero elements
    int cholesky_v_m[cholesky_n_m];       // sparse cholesky in magnitude - columns of nonzero elements
    int cholesky_u_m[M+1];                // sparse cholesky in magnitude - where in w each row starts
    int cholesky_n_c;                     // sparse cholesky in colour - number of nonzero elements
    vector[cholesky_n_c] cholesky_w_c;    // sparse cholesky in colour - nonzero elements
    int cholesky_v_c[cholesky_n_c];       // sparse cholesky in colour - columns of nonzero elements
    int cholesky_u_c[C+1];                // sparse cholesky in colour - where in w each row starts
}
parameters {
    matrix[M_subspace,C_subspace] z[S];
}
transformed parameters {

    vector[P] x[M,C]; // Probability in logit-space
    
    // Loop over magnitude and colour
    for (m in 1:M){
        for (c in 1:C){
            
            // Local variable
            vector[S] b;
                
            // Compute b
            for (s in 1:S){
                b[s] = mu[s] + sigma[s] * (cholesky_w_m[cholesky_u_m[m]:cholesky_u_m[m+1]-1] * z[s,cholesky_v_m[cholesky_u_m[m]:cholesky_u_m[m+1]-1], cholesky_v_c[cholesky_u_c[c]:cholesky_u_c[c+1]-1]] * cholesky_w_c[cholesky_u_c[c]:cholesky_u_c[c+1]-1]);
            }
                
            // Compute x
            x[m,c] = csr_matrix_times_vector(P, S, wavelet_w, wavelet_v, wavelet_u, b);

        }  
    }
    
}
model {

    // Prior
    for (s in 1:S){
        to_vector(z[s]) ~ std_normal();
    }
    
    // Likelihood
    for (m in 1:M){
        for (c in 1:C){
            k[m,c] ~ binomial_logit(n[m,c], x[m,c]);
        }
    }
    
}

In [ ]:
        # Load in sparse matrix
        nmodes = 0
        for j in self.j:
            if j==-1: nmodes += 1
            else: nmodes += 12*(2**j)**2
        npix = self.nside_to_npix(self.nside)
        n = len(mag)
        x = np.zeros(n)

        @njit
        def matrix_multiply(x, b, KM, KC, n, H, M, C, wavelet_w, wavelet_v, wavelet_u, pix):

            # Iterate over pixels
            for i, ipix in enumerate(pix):
                # Iterate over modes which are not sparsified in Y
                for iY, iH in enumerate(wavelet_v[wavelet_u[ipix]:wavelet_u[ipix+1]]):
                    x[i] += np.dot(np.dot(KM[i], b[iH]), KC[i]) * wavelet_w[wavelet_u[ipix]+iY]

        # Contstruct covariance kernel for new positions.
        KmM = self.covariance_kernel(mag, self.Mcenters, lengthscale=self.lengthscale)
        KcC = self.covariance_kernel(color, self.Ccenters, lengthscale=self.lengthscale)

        matrix_multiply(x, self.b, (KmM @ self._inv_KMM), (KcC @ self._inv_KCC), n, self.H, self.M, self.C,\
                  self.basis['wavelet_w'], self.basis['wavelet_v'], self.basis['wavelet_u'], pix)